In [1]:
suppressPackageStartupMessages({
    library(dplyr)
    library(ggplot2)
    library(biomaRt)
    library(dplyr)
})

In [24]:
# Counts
counts <- read.table("CCLE_RNAseq_rsem_genes_tpm_20180929.txt", header = TRUE, row.names = 1) %>%
    dplyr::select(THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE)
rownames(counts) <- gsub("\\..*", "", rownames(counts))
counts <- as.matrix(counts)
head(counts)

,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
ENSG00000000003,1.40
ENSG00000000005,0.00
ENSG00000000419,55.70
ENSG00000000457,4.54
ENSG00000000460,18.05
ENSG00000000938,7.24


In [16]:
# mart <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
ensembl_annot <- getBM(attributes=c("ensembl_gene_id","external_gene_name"),
                       filters = "ensembl_gene_id",
                       values = rownames(counts), 
                       mart = mart)
head(ensembl_annot)


Batch submitting query [==>----------------------------]   8% eta:  2m

Batch submitting query [====>--------------------------]  17% eta:  2m

Batch submitting query [=======>-----------------------]  25% eta:  2m

Batch submitting query [=========>---------------------]  33% eta:  2m

Batch submitting query [============>------------------]  42% eta:  1m

Batch submitting query [===============>---------------]  50% eta:  1m

Batch submitting query [=================>-------------]  58% eta: 44s

Batch submitting query [====================>----------]  67% eta: 34s

Batch submitting query [======================>--------]  75% eta: 25s

Batch submitting query [=========================>-----]  83% eta: 20s

Batch submitting query [===========================>---]  92% eta:  9s
                                                                      



,ensembl_gene_id,external_gene_name
,<chr>,<chr>
1,ENSG00000000003,TSPAN6
2,ENSG00000000005,TNMD
3,ENSG00000000419,DPM1
4,ENSG00000000457,SCYL3
5,ENSG00000000460,FIRRM
6,ENSG00000000938,FGR


In [28]:
cols <- as.data.frame(rownames(counts))
new_names <- left_join(cols, ensembl_annot, by = c("rownames(counts)" = "ensembl_gene_id"))
rownames(counts) <- new_names$external_gene_name
count_df <- as.data.frame(counts)
head(count_df)

,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
,<dbl>
TSPAN6,1.40
TNMD,0.00
DPM1,55.70
SCYL3,4.54
FIRRM,18.05
FGR,7.24


In [29]:
genes <- c(
  "LILRB1",
  "LILRB2",
  "LILRB3",
  "LAIR1",
  "HLA-G",
  "HAVCR2",   # TIM3
  "CD33",     # SIGLEC3
  "SIGLEC7",
  "SIGLEC9",
  "SIRPA",
  "LILRB3",
  "LILRB4",
  "LILRB5",
  "LILRA1",
  "LILRA2",
  "LILRA3",
  "LILRA4",
  "LILRA5",
  "LILRA6"
)

In [34]:
filtered_df <- count_df[rownames(count_df) %in% genes, ]
filtered_df

[1]   2.26   1.29  81.88   0.00   0.74   1.26   0.25 241.00   5.33  79.26
[11]   0.04  26.34   0.21   0.42  43.50   0.00

In [46]:
filtered_df <- count_df %>%
    dplyr::filter(rownames(.) %in% genes)%>%
    dplyr::mutate(is_lil = grepl("^LIL", rownames(.))) %>% 
    dplyr::arrange(desc(is_lil), rownames(.)) %>%
    dplyr::select(-is_lil) %>%
    dplyr::rename(`THP1 Counts (tpm)` = THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE)
filtered_df

,THP1 Counts (tpm)
,<dbl>
LILRA1,1.29
LILRA2,43.50
LILRA4,0.42
LILRA5,0.04
LILRA6,0.00
LILRB1,2.26
LILRB2,1.26
LILRB3,0.21
LILRB4,79.26


In [45]:
write.table(filtered_df, "result.csv", sep = ",", row.names = TRUE)